# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [9]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [10]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a empty list to get the list of files
file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = file_path_list + glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [11]:
# print (file_path_list)
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
#print (file_path_list)
for f in file_path_list:
    #print ('The file path' + f + 'is read now')  

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)
        #print (next(csvreader))
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
          
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [12]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

7443


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [13]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [14]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
# Set KEYSPACE to the keyspace specified above
# Connecting to our keyspace:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



In [16]:
# song_info_by_session table is created for Query 1 to retrieve the artist, song title and song's length which was heard during sessionId = 338 and
# itemInSession = 4. itemInSession is the primary key and sesionId is the clustering key as the 
# WHERE caluse in query is centered around itemInSession and sessionId

query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(itemInSession int, sessionId int, artist text, song text, length float, PRIMARY KEY (itemInSession, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)          

In [17]:
# Load the records into the song_info_by_session table for executing the Query2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (itemInSession, sessionId , artist , song , length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
# Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
# SELECT artist, song title and song's length in the song_info_by_session table where 
# sessionId = 338, and itemInSession = 4

query = "Select artist, song, length from song_info_by_session where itemInSession = 4 and sessionId = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
  print ('Artist : ' + row.artist, ', Song : ' + row.song, ', Song length : ' +  str(row.length))

Artist : Faithless , Song : Music Matters (Mark Knight Dub) , Song length : 495.30731201171875


In [19]:
# song_info_by_userId table is created to execute the Query 2 to retrieve the name of the artist, song(sorted by itemInSession) and user(first and last name)
# for userid = 10, sessionid = 182. userId and sessionId together makes the primary key as the query is extracted on userid and sessionId 
# in the WHERE clause and the composite key is itemInSession as the song needs to be sorted on itemInSession.

query = "CREATE TABLE IF NOT EXISTS song_info_by_userId "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [20]:
# Load the records into the song_info_by_userId table for executing the Query2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_userId (userId, sessionId ,itemInSession , artist , song , firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
# Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [21]:
#  SELECT artist, song (sorted by itemInSession) and user (first and last name)
#  for userid = 10, sessionid = 182 from the song_info_by_userId table

query = "Select artist, song, firstName, lastName FROM song_info_by_userId where userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print ('Artist : ' + row.artist, ', Song : ' + row.song, ', User : ' +  row.firstname + ' '+ row.lastname)

Artist : Down To The Bone , Song : Keep On Keepin' On , User : Sylvie Cruz
Artist : Three Drives , Song : Greece 2000 , User : Sylvie Cruz
Artist : Sebastien Tellier , Song : Kilometer , User : Sylvie Cruz
Artist : Lonnie Gordon , Song : Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , User : Sylvie Cruz


In [22]:
# user_info_by_song table is created to execute the Query 3 to retrieve the name of the user(first and last name) who listened to the 
# song 'All Hands Against His Own'. song is made the primary key of the table as the results are extracted on songs and userId is added as composite
# key to make the rows unique.

query = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query = query + "(song text, userId int,firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                      

In [23]:
# Load the records into the user_info_by_song table for executing the Query3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_info_by_song ( song , userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, ( line[9], int(line[10]), line[1], line[4]))

In [24]:
## SELECT name (first and last) from user_info_by_song table who listened to the song 'All Hands Against His Own'

query = "Select firstName, lastName FROM user_info_by_song where song = 'All Hands Against His Own'"
count = 0
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print ('User : ' +  row.firstname + ' '+ row.lastname)
    count = count + 1
print ('There are totally ' + str(count) + ' users listened to the song "All Hands Against His Own"')   

User : Jacqueline Lynch
User : Tegan Levine
User : Sara Johnson
There are totally 3 users listened to the song "All Hands Against His Own"


### Drop the tables before closing out the sessions

In [25]:
## Drop the table before closing out the sessions

# DROP song_info_by_session table used by QUERY 1
query = "Drop Table song_info_by_session"
try:
    rows = session.execute(query)
    print ('The table song_info_by_session deleted')
except Exception as e:
    print(e)
    

# DROP song_info_by_userId table used by QUERY 2
query = "Drop Table song_info_by_userId"
try:
    rows = session.execute(query)
    print ('The table song_info_by_userId deleted')
except Exception as e:
    print(e)
    
    
# DROP user_song_info table used by QUERY 3
query = "Drop Table user_info_by_song"
try:
    rows = session.execute(query)
    print ('The table user_info_by_song deleted')
except Exception as e:
    print(e)
    

The table song_info_by_session deleted
The table song_info_by_userId deleted
The table user_info_by_song deleted


### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()